In [4]:
# import important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("FinalBalancedDataset.csv")

In [6]:
df.head()

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [7]:
df.tail()

,Unnamed: 0,Toxicity,tweet
56740,56740,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
56741,56741,1,"you've gone and broke the wrong heart baby, an..."
56742,56742,1,young buck wanna eat!!.. dat nigguh like I ain...
56743,56743,1,youu got wild bitches tellin you lies
56744,56744,0,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56745 non-null  int64 
 1   Toxicity    56745 non-null  int64 
 2   tweet       56745 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [10]:
df.isnull().sum()

Unnamed: 0    0
Toxicity      0
tweet         0
dtype: int64

In [11]:
# drop unwanted column like 'Unmaned'

df = df.drop(['Unnamed: 0'], axis=1)

In [12]:
df.head()

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [13]:
df['Toxicity'].value_counts()

Toxicity
0    32592
1    24153
Name: count, dtype: int64

# Main code

In [14]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
import re

[nltk_data] Downloading package punkt to C:\Users\Sanket
[nltk_data]     Pawar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Sanket
[nltk_data]     Pawar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sanket
[nltk_data]     Pawar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sanket
[nltk_data]     Pawar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sanket Pawar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Text Processing

In [15]:
wordnet_lemmatizer = WordNetLemmatizer()

In [17]:
def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    
    # Clean the text by removing non-alphabetic characters and tokenizing
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.split()
    text = ' '.join(text)
    text = word_tokenize(text)
    
    # POS tagging
    text = nltk.pos_tag(text)

    # Lemmatization
    lemma = []
    for i in text:
        lemma.append(wordnet_lemmatizer.lemmatize(i[0], get_wordnet_pos(i[1])))
    
    # Return the lemmatized text
    return ' '.join(lemma)

# Example usage
input_text = "The striped bats are hanging on their feet for best."
output = prepare_text(input_text)
print(output)

The striped bat be hang on their foot for best


In [18]:
df['Clean_tweets'] = df['tweet'].apply(prepare_text)

In [19]:
df.head()

,Toxicity,tweet,Clean_tweets
0,0,@user when a father is dysfunctional and is s...,user when a father be dysfunctional and be so ...
1,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t use c...
2,0,bihday your majesty,bihday your majesty
3,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,0,factsguide: society now #motivation,factsguide society now motivation


# tf-idf for features

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [22]:
corpus = df['Clean_tweets'].values.astype('U')

In [23]:
stopwords = list(nltk_stopwords.words('english'))

In [24]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

In [25]:
tf_idf = count_tf_idf.fit_transform(corpus)

In [26]:
import pickle

In [27]:
pickle.dump(count_tf_idf, open("tf_idf.pkl", "wb"))

In [28]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(
    tf_idf, df['Toxicity'], test_size=0.8, random_state=42, shuffle = True)

# Create binary classification model

In [29]:
model_bayes = MultinomialNB()

In [30]:
model_bayes = model_bayes.fit(tf_idf_train, target_train)

In [31]:
y_pred_proba = model_bayes.predict_proba(tf_idf_test)[::,1]

In [32]:
y_pred_proba

array([0.90208069, 0.27944795, 0.79050675, ..., 0.09423598, 0.20495379,
       0.32128745])

In [33]:
fpr,tpr,_ = roc_curve(target_test, y_pred_proba)

In [34]:
final_roc_auc = roc_auc_score(target_test, y_pred_proba)

In [36]:
print(final_roc_auc)

0.9659576344417777


In [37]:
test_text="I hate you moron"
test_tfidf = count_tf_idf.transform([test_text])
display(model_bayes.predict_proba(test_tfidf))
display(model_bayes.predict(test_tfidf))

array([[0.39929905, 0.60070095]])

array([1])

# save model

In [38]:
pickle.dump(model_bayes, open("Toxicity_Sentiment_model.pkl", "wb"))